In [ ]:
import numpy as np
from sklearn.svm import SVC
import sklearn.model_selection as model_selection

In [ ]:
def load_data(file_path):
  data = np.loadtxt(file_path)
  X = data[:, 1:]
  Y = data[:, 0]
  return X, Y

def create_binary_labels(y, target_digit):
  return np.where(y == target_digit, 1, -1)

def train_and_evaluate(trainX, trainY, C, Q, test=None):
  svm = SVC(C=C, kernel='poly', degree=Q, coef0=1, gamma=1)
  svm.fit(trainX, trainY)

  predictions = svm.predict(trainX)
  e_in = np.mean(predictions != trainY)
  sv = len(svm.support_)

  if test:
    predictions = svm.predict(test[0])
    e_out = np.mean(predictions != test[1])
    return e_in, sv, e_out

  return e_in, sv, None

def get_support_vectors(X, Y, digit, C, Q):
  binY = create_binary_labels(Y, digit)
  svm = SVC(C=C, kernel='poly', degree=Q, coef0=1, gamma=1)
  svm.fit(X, binY)

  return sum(svm.n_support_)

def evaluate_classifiers(X, Y, classifiers, C, Q, test=None):
  for classifier in classifiers:
    y_train_binary = create_binary_labels(Y, classifier)
    e_in, sv, e_out = train_and_evaluate(X, y_train_binary, C, Q, test=test)
    print(f"{classifier}: e_in = {e_in}")

In [ ]:
train_file_path = "/content/features.train"
test_file_path = "/content/features.test"

trainX, trainY = load_data(train_file_path)
testX, testY = load_data(test_file_path)

In [ ]:
C = 0.01
Q = 2

evaluate_classifiers(trainX, trainY, range(0, 10, 2), C, Q)
print("----------------------")
evaluate_classifiers(trainX, trainY, range(1, 11, 2), C, Q)

0: e_in = 0.10588396653408312
2: e_in = 0.10026059525442327
4: e_in = 0.08942531888629818
6: e_in = 0.09107118365107666
8: e_in = 0.07433822520916199
----------------------
1: e_in = 0.014401316691811822
3: e_in = 0.09024825126868742
5: e_in = 0.07625840076807022
7: e_in = 0.08846523110684405
9: e_in = 0.08832807570977919


In [ ]:
sv1 = get_support_vectors(trainX, trainY, 0, C, Q)
sv2 = get_support_vectors(trainX, trainY, 1, C, Q)

sv_diff = abs(sv1 - sv2)
print(f"Difference in support vectors: {sv_diff}")

Difference in support vectors: 1793


In [ ]:
def filter_data_15(X, y):
  filteredX = X[(y == 1) | (y == 5)]
  filteredY = y[(y == 1) | (y == 5)]
  binY = np.where(filteredY == 1, 1, -1)
  return filteredX, binY

filteredX, filteredY = filter_data_15(trainX, trainY)

C_values = [.001, .01, .1, 1]
Q_values = [2, 5]

for Q in Q_values:
  for C in C_values:
    ein, sv, e_out = train_and_evaluate(filteredX, filteredY, C, Q, test=(testX, testY))
    print(f"Q = {Q}, C = {C}: Ein = {ein}, E_out = {e_out}, Support Vectors = {sv}")

Q = 2, C = 0.001: Ein = 0.004484304932735426, E_out = 0.8719481813652217, Support Vectors = 76
Q = 2, C = 0.01: Ein = 0.004484304932735426, E_out = 0.872446437468859, Support Vectors = 34
Q = 2, C = 0.1: Ein = 0.004484304932735426, E_out = 0.872446437468859, Support Vectors = 24
Q = 2, C = 1: Ein = 0.0032030749519538757, E_out = 0.872446437468859, Support Vectors = 24
Q = 5, C = 0.001: Ein = 0.004484304932735426, E_out = 0.872446437468859, Support Vectors = 25
Q = 5, C = 0.01: Ein = 0.003843689942344651, E_out = 0.872446437468859, Support Vectors = 23
Q = 5, C = 0.1: Ein = 0.0032030749519538757, E_out = 0.872446437468859, Support Vectors = 25
Q = 5, C = 1: Ein = 0.0032030749519538757, E_out = 0.8729446935724963, Support Vectors = 21


In [ ]:
C_values = [.0001, .001, .01, .1, 1]
Q = 2
runs = 100
folds = 10

fold = model_selection.RepeatedStratifiedKFold(n_splits=folds, n_repeats=runs)
best_C = []

for train_index, val_index in fold.split(filteredX, filteredY):
    trainX, valX = filteredX[train_index], filteredX[val_index]
    trainY, valY = filteredY[train_index], filteredY[val_index]

    best_score = 0
    best = None

    for C in C_values:
        svm = SVC(kernel='poly', C=C, degree=Q, gamma=1, coef0=1)
        svm.fit(trainX, trainY)
        score = svm.score(valX, valY)
        if score > best_score:
            best_score = score
            best = C

    best_C.append(best)

_, count = np.unique(best_C, return_counts=True)

for i in range(len(C_values)):
  print(f"C {C_values[i]}: {count[i]}")

C 0.0001: 387
C 0.001: 502
C 0.01: 52
C 0.1: 28
C 1: 31


In [ ]:
filteredX_test, filteredY_test = filter_data_15(testX, testY)

def rbf(C):
    svm = SVC(kernel='rbf', C=C, degree=Q, gamma=1)
    svm.fit(filteredX, filteredY)
    predictions = svm.predict(filteredX)
    e_in = np.mean(predictions != filteredY)
    predictions = svm.predict(filteredX_test)
    e_out = np.mean(predictions != filteredY_test)
    print(f"C {C}, e_in = {e_in}, e_out = {e_out}")

C_values = [.01, 1, 100, 1e4, 1e6]

for C in C_values:
  rbf(C)

C 0.01, e_in = 0.003843689942344651, e_out = 0.02358490566037736
C 1, e_in = 0.004484304932735426, e_out = 0.02122641509433962
C 100, e_in = 0.0032030749519538757, e_out = 0.018867924528301886
C 10000.0, e_in = 0.0025624599615631004, e_out = 0.02358490566037736
C 1000000.0, e_in = 0.0006406149903907751, e_out = 0.02358490566037736
